# Hierarchical Clustering - Mean Shift

K-Means is called "flat clustering" and required us to specify how many clusters we think exists

Mean shift estimates how many clusters exists and then finds the centers of those clusters

Mean Shift (MS) starts by placing a cluster center at the "1st" or "1st random" feature point.
- It then  assigns a "bandwidth" or "radius" around each initial cluster center and determines how many features are within the "bandwidth" or "radius".
    - This is done for every single "cluster center".

After taking the mean of all of the data points in the cluster center + radius (bandwidth), it assigns this average as the new cluster center.
    - MS again assigned a bandwidth around that new cluster center and iterates until convergence 
       - convergence means difference between old and new cluster center is below epsilon

As the cluster center moves around looking for convergence, it finds new feature points.  
These new features points change the cluster center (maybe they should change the bandwidth too(?).

The method allows such that if you start from any point any given cluster, MS will find the same set of feature points within the same cluster (cluster center, radius).  

Once MS finds no new feature points and stops moving the cluster center, it has reached convergence.

But there are known feature points outside of this cluster.  This must start again using the feature points that are outside the given cluster.
- This creates a new cluster (i.e. MS determines the number of clusters && their centers)
    
This new cluster iterates to convergence again.  If there are more more feature points outside BOTH clusters, then MS will make a third cluster.

If all feature points are accounted for by known clusters with centers and bandwidths, then MS has completed optimization

---
What if the clustering is not obvious?
---

MS can make sets of concentric bandwidths around the same clustering center, but with weights assigned to them.
Such that the set of feature sets has 1 given center, but the points furthest away from that center are weighted less than the feature points that are closest to the center.
- for a 'cluster' with 3 radii, the weights could be (3,2,1) for (nearest, middlest, furthest), respectively.
- could also penalize in some sort of squared error fashion too

---
Simple Example of Hierarchical Clustering with Mean Shift
---

---
Titanic Data Set
---

We assumed that the titanic data set would split into 2 groups because either survived or died; and we assumed that the more wealthy a passenger was, the more likely they were to have survived

Using MeanShift instead of K-Means lets us remove that assumption and ask not only "where" are the cluster centers, but how many are there.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

from sklearn import preprocessing, cross_validation
from sklearn.cluster import MeanShift

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import style
style.use('fivethirtyeight')

/Users/jonathanfraine/anaconda/lib/python3.5/site-packages/matplotlib/__init__.py:1028: UserWarning: Illegal line #312
	"                               legend, else a rectangle
"
	in file "/Users/jonathanfraine/anaconda/lib/python3.5/site-packages/matplotlib/mpl-data/matplotlibrc"
  warnings.warn('Illegal %s' % error_details)


In [10]:
'''
Pclass Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
survival Survival (0 = No; 1 = Yes)
name Name
sex Sex
age Age
sibsp Number of Siblings/Spouses Aboard
parch Number of Parents/Children Aboard
ticket Ticket Number
fare Passenger Fare (British pound)
cabin Cabin
embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
boat Lifeboat
body Body Identification Number
home.dest Home/Destination
'''


# https://pythonprogramming.net/static/downloads/machine-learning-data/titanic.xls
df = pd.read_excel('titanic.xls')
original_df = pd.DataFrame.copy(df)

In [11]:
df.drop(['body','name'], 1, inplace=True)
df.fillna(0,inplace=True)

In [12]:
def handle_non_numerical_data(df):
    
    # handling non-numerical data: must convert.
    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        #print(column,df[column].dtype)
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            
            column_contents = df[column].values.tolist()
            #finding just the uniques
            unique_elements = set(column_contents)
            # great, found them. 
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    # creating dict that contains new
                    # id per unique string
                    text_digit_vals[unique] = x
                    x+=1
            # now we map the new "id" vlaue
            # to replace the string. 
            df[column] = list(map(convert_to_int,df[column]))

    return df

In [13]:
df = handle_non_numerical_data(df)
df.drop(['ticket','home.dest'], 1, inplace=True)

In [14]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

In [15]:
ms = MeanShift()
ms.fit(X)

MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [16]:
labels          = ms.labels_
cluster_centers = ms.cluster_centers_
n_clusters      = len(cluster_centers)
n_clusters

4

In [17]:
original_df['cluster_group'] = np.nan

for i,label in enumerate(labels):
    original_df['cluster_group'].iloc[i] = label

n_clusters_ = len(np.unique(labels))
n_clusters_

/Users/jonathanfraine/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


4

In [21]:
survival_rates = {}
for clusterNow in range(n_clusters_):
    temp_df = original_df[(original_df['cluster_group'] == float(clusterNow))]
    
    # Define Survival Clusters
    survival_cluster = temp_df[(temp_df['survived'] == 1)]
    survival_rate    = len(survival_cluster) / len(temp_df)
    survival_rates[clusterNow] = survival_rate

survival_rates

{0: 0.3685897435897436, 1: 1.0, 2: 0.7391304347826086, 3: 0.1}

In [30]:
for clusterNow in range(n_clusters_):
    # print(clusterNow, original_df[original_df['cluster_group'] == clusterNow]['pclass'].drop_duplicates().values, survival_rates[clusterNow]*100, end='\n\n')
    print('Cluster: ', clusterNow)
    print(clusterNow, original_df.describe(), end='\n\n')

Cluster:  0
0             pclass     survived          age        sibsp        parch  \
count  1309.000000  1309.000000  1046.000000  1309.000000  1309.000000   
mean      2.294882     0.381971    29.881135     0.498854     0.385027   
std       0.837836     0.486055    14.413500     1.041658     0.865560   
min       1.000000     0.000000     0.166700     0.000000     0.000000   
25%       2.000000     0.000000          NaN     0.000000     0.000000   
50%       3.000000     0.000000          NaN     0.000000     0.000000   
75%       3.000000     1.000000          NaN     1.000000     0.000000   
max       3.000000     1.000000    80.000000     8.000000     9.000000   

              fare        body  cluster_group  
count  1308.000000  121.000000    1309.000000  
mean     33.295479  160.809917       0.097021  
std      51.758668   97.696922       0.451534  
min       0.000000    1.000000       0.000000  
25%            NaN         NaN       0.000000  
50%            NaN         NaN 

/Users/jonathanfraine/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [ ]:
colors = 10*plt.rcParams['axes.color_cycle']

fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in range(len(X)):
    ax.scatter(X[i][0], X[i][1], X[i][6], c=colors[labels[i]], marker='o')

ax.scatter(cluster_centers[:,0], cluster_centers[:,1], cluster_centers[:,3], 
               marker='x', color='k', s=150, lw=5, zorder=10)

In [ ]:
for k in np.unique(labels):
    print(np.where(labels == k)[0].sum())